In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [2]:
datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)

In [11]:
data_dir = './images'

In [12]:
train_data = datagen.flow_from_directory(data_dir, target_size = (250,250), batch_size = 32, class_mode = 'categorical',
                                        subset = 'training')

val_data = datagen.flow_from_directory(data_dir, target_size = (250,250), batch_size = 32, class_mode = 'categorical',
                                        subset = 'validation')

Found 128255 images belonging to 145 classes.
Found 32031 images belonging to 145 classes.


In [13]:
ResNet_V2_50 = 'https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5'

In [14]:
import tensorflow_hub as hub

In [15]:
model_ResNet = tf.keras.Sequential([
    hub.KerasLayer(ResNet_V2_50, trainable = False, input_shape = (250,250,3), name = 'Resnet_V2_50'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(145, activation = 'softmax', name = 'Output_layer')
])

model_ResNet.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

In [16]:
model_ResNet.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Resnet_V2_50 (KerasLayer)   (None, 1001)              25615849  
                                                                 
 flatten_1 (Flatten)         (None, 1001)              0         
                                                                 
 Output_layer (Dense)        (None, 145)               145290    
                                                                 
Total params: 25,761,139
Trainable params: 145,290
Non-trainable params: 25,615,849
_________________________________________________________________


In [17]:
resnet_model = model_ResNet.fit(train_data, epochs = 10, verbose = 1)

 915/4008 [=====>........................] - ETA: 5:56 - loss: 2.6843 - accuracy: 0.3742

In [ ]:
model_ResNet.evaluate(val_data)

In [ ]:
model_name = 'ResNetV2_145class.h5'
model_ResNet.save(model_name, save_format='h5')